<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#База" data-toc-modified-id="База-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>База</a></span></li><li><span><a href="#Проход-по-страницам" data-toc-modified-id="Проход-по-страницам-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Проход по страницам</a></span></li><li><span><a href="#Проход-по-страницам-объектов" data-toc-modified-id="Проход-по-страницам-объектов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проход по страницам объектов</a></span></li><li><span><a href="#Умные-трюки-для-парсинга" data-toc-modified-id="Умные-трюки-для-парсинга-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Умные трюки для парсинга</a></span></li><li><span><a href="#Еще-какой-то-код" data-toc-modified-id="Еще-какой-то-код-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Еще какой-то код</a></span><ul class="toc-item"><li><span><a href="#авито" data-toc-modified-id="авито-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>авито</a></span></li><li><span><a href="#циан" data-toc-modified-id="циан-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>циан</a></span></li></ul></li></ul></div>

## База

In [1]:
# пригодится далее
! pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=eb364872f55e3918f8b108f9c80acb16ab2a284cc0ffe67d9c1a73c7e126e233
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [1]:
# основные библиотеки для парсинга
import requests  
from bs4 import BeautifulSoup

In [3]:
# делаем запрос на сайт
url = 'http://books.toscrape.com/catalogue/page-1.html'
response = requests.get(url)
response


<Response [200]>

In [4]:
# вот так выглядит HTML код сайта
response.content[:1000]

b'\n\n<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:30" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon"

In [5]:
# достаем структуру сайта
tree = BeautifulSoup(response.content, 'html.parser')
# tree

In [6]:
# нашли по тегу title название книги
tree.title.text.strip()

'All products | Books to Scrape - Sandbox'

In [7]:
# нашли первый h3 
tree.find('h3')

<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

In [8]:
# здесь по тегу h3 нашли все книги на странице
tree.find_all('h3')

[<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>,
 <h3><a href="tipping-the-velvet_999/index.html" title="Tipping the Velvet">Tipping the Velvet</a></h3>,
 <h3><a href="soumission_998/index.html" title="Soumission">Soumission</a></h3>,
 <h3><a href="sharp-objects_997/index.html" title="Sharp Objects">Sharp Objects</a></h3>,
 <h3><a href="sapiens-a-brief-history-of-humankind_996/index.html" title="Sapiens: A Brief History of Humankind">Sapiens: A Brief History ...</a></h3>,
 <h3><a href="the-requiem-red_995/index.html" title="The Requiem Red">The Requiem Red</a></h3>,
 <h3><a href="the-dirty-little-secrets-of-getting-your-dream-job_994/index.html" title="The Dirty Little Secrets of Getting Your Dream Job">The Dirty Little Secrets ...</a></h3>,
 <h3><a href="the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html" title="The Coming Woman: A Novel Based on the Life of the Infamous Femi

In [9]:
len(tree.find_all('h3'))

20

In [10]:
# взяли первую книгу и достали её ссылку через тег a и атрибут href
tree.find_all('h3')[0].a.get('href')

'a-light-in-the-attic_1000/index.html'

In [11]:
# взяли первую книгу и достали её название через тег a и атрибут title
tree.find_all('h3')[0].a.get('title')

'A Light in the Attic'

## Проход по страницам

In [12]:
# все книжки на одной странице
books_on_one_page = tree.find_all('h3')

In [13]:
# загрузим полезные модули
from collections import defaultdict as dd

# из pandas берем удобные функции для таблиц. можно не использовать её
import pandas as pd

# визуализация циклов
from tqdm.notebook import tqdm

import time
import os
from fake_useragent import UserAgent

In [14]:
# парсим одну страницу
d = dd(lambda:{})
for book in books_on_one_page:
    href = book.a.get('href')
    title = book.a.get('title')
    d[href] = {'title':title}
dict(d)

{'a-light-in-the-attic_1000/index.html': {'title': 'A Light in the Attic'},
 'its-only-the-himalayas_981/index.html': {'title': "It's Only the Himalayas"},
 'libertarianism-for-beginners_982/index.html': {'title': 'Libertarianism for Beginners'},
 'mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html': {'title': 'Mesaerion: The Best Science Fiction Stories 1800-1849'},
 'olio_984/index.html': {'title': 'Olio'},
 'our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html': {'title': 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991'},
 'rip-it-up-and-start-again_986/index.html': {'title': 'Rip it Up and Start Again'},
 'sapiens-a-brief-history-of-humankind_996/index.html': {'title': 'Sapiens: A Brief History of Humankind'},
 'scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html': {'title': "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)"},
 'set-me-free_988/index.html': {'title'

In [15]:
# парсим 50 страниц
d = dd(lambda:{})
for page in range(1,51):
    url = f'http://books.toscrape.com/catalogue/page-{page}.html'
    response = requests.get(url)
    tree = BeautifulSoup(response.content, 'html.parser')
    books_on_one_page = tree.find_all('h3')
    for book in books_on_one_page:
        href = book.a.get('href')
        title = book.a.get('title')
        d[href] = {'title':title}

In [16]:
# получаем табличку
pd.DataFrame(d).T

,title
a-light-in-the-attic_1000/index.html,A Light in the Attic
tipping-the-velvet_999/index.html,Tipping the Velvet
soumission_998/index.html,Soumission
sharp-objects_997/index.html,Sharp Objects
sapiens-a-brief-history-of-humankind_996/index.html,Sapiens: A Brief History of Humankind
...,...
alice-in-wonderland-alices-adventures-in-wonderland-1_5/index.html,Alice in Wonderland (Alice's Adventures in Won...
ajin-demi-human-volume-1-ajin-demi-human-1_4/index.html,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)"
a-spys-devotion-the-regency-spies-of-london-1_3/index.html,A Spy's Devotion (The Regency Spies of London #1)
1st-to-die-womens-murder-club-1_2/index.html,1st to Die (Women's Murder Club #1)


## Проход по страницам объектов

In [17]:
url = 'https://books.toscrape.com/catalogue/how-music-works_979/index.html'
response = requests.get(url)
tree = BeautifulSoup(response.content, 'html.parser')

In [18]:
# <p class="price_color">£37.32</p>
# ищем цену
tree.find('p', {'class':'price_color'}).text

'£37.32'

In [19]:
# <p class="instock availability">
# ищем сколько в наличии
tree.find('p', {'class':'instock availability'}).text.strip()

'In stock (19 available)'

In [20]:
# <div class="col-sm-6 product_main">
# рейтинг
main_info_subtree = tree.find('div', {'class':'col-sm-6 product_main'})
main_info_subtree.find_all('p')[2].get('class')[1]

'Two'

In [21]:
# описание книги
tree.find_all('p')[3].text

'How Music Works is David Byrne’s remarkable and buoyant celebration of a subject he has spent a lifetime thinking about. In it he explores how profoundly music is shaped by its time and place, and he explains how the advent of recording technology in the twentieth century forever changed our relationship to playing, performing, and listening to music.Acting as historian an How Music Works is David Byrne’s remarkable and buoyant celebration of a subject he has spent a lifetime thinking about. In it he explores how profoundly music is shaped by its time and place, and he explains how the advent of recording technology in the twentieth century forever changed our relationship to playing, performing, and listening to music.Acting as historian and anthropologist, raconteur and social scientist, he searches for patterns—and shows how those patterns have affected his own work over the years with Talking Heads and his many collaborators, from Brian Eno to Caetano Veloso. Byrne sees music as p

In [22]:
# выгружаем табличку
# tree.find('table') # тут таблица
# передаем её в код ниже и просто получаем словарь...
pd.read_html(str(tree.find('table')))[0].set_index(0).to_dict()[1]

{'Availability': 'In stock (19 available)',
 'Number of reviews': '0',
 'Price (excl. tax)': '£37.32',
 'Price (incl. tax)': '£37.32',
 'Product Type': 'Books',
 'Tax': '£0.00',
 'UPC': '327f68a59745c102'}

In [23]:
# собираем всё в кучку
price = tree.find('p', {'class':'price_color'}).text
price = float(price[1:])

instock_count = tree.find('p', {'class':'instock availability'}).text
instock_count = int(instock_count.strip().split('(')[1].split(' ')[0])

main_info_subtree = tree.find('div', {'class':'col-sm-6 product_main'})
stars = main_info_subtree.find_all('p')[2].get('class')[1]

description = tree.find_all('p')[3].text

info = pd.read_html(str(tree.find('table')))[0].set_index(0).to_dict()[1]

info['price']=price
info['instock_count']=instock_count
info['stars']=stars
info['description']=description

info

{'Availability': 'In stock (19 available)',
 'Number of reviews': '0',
 'Price (excl. tax)': '£37.32',
 'Price (incl. tax)': '£37.32',
 'Product Type': 'Books',
 'Tax': '£0.00',
 'UPC': '327f68a59745c102',
 'description': 'How Music Works is David Byrne’s remarkable and buoyant celebration of a subject he has spent a lifetime thinking about. In it he explores how profoundly music is shaped by its time and place, and he explains how the advent of recording technology in the twentieth century forever changed our relationship to playing, performing, and listening to music.Acting as historian an How Music Works is David Byrne’s remarkable and buoyant celebration of a subject he has spent a lifetime thinking about. In it he explores how profoundly music is shaped by its time and place, and he explains how the advent of recording technology in the twentieth century forever changed our relationship to playing, performing, and listening to music.Acting as historian and anthropologist, raconteu

In [24]:
# собираем всё в фунцию
def parse_one_book(tree):
    price = tree.find('p', {'class':'price_color'}).text
    price = float(price[1:])

    instock_count = tree.find('p', {'class':'instock availability'}).text
    instock_count = int(instock_count.strip().split('(')[1].split(' ')[0])

    main_info_subtree = tree.find('div', {'class':'col-sm-6 product_main'})
    stars = main_info_subtree.find_all('p')[2].get('class')[1]

    description = tree.find_all('p')[3].text

    info = pd.read_html(str(tree.find('table')))[0].set_index(0).to_dict()[1]

    info['price']=price
    info['instock_count']=instock_count
    info['stars']=stars
    info['description']=description
    
    return info

In [25]:
base_url = 'https://books.toscrape.com/catalogue/'

# начинаем проход по всем страницам
# список страницы мы опредилили ранее
for book_url in tqdm(d):
    url = base_url+book_url
    response = requests.get(url)
    tree = BeautifulSoup(response.content, 'html.parser')
    results = parse_one_book(tree)
    d[book_url].update(results)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [26]:
# получаем вот такую таблицу
pd.DataFrame(d).T

,title,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews,price,instock_count,stars,description
a-light-in-the-attic_1000/index.html,A Light in the Attic,a897fe39b1053632,Books,£51.77,£51.77,£0.00,In stock (22 available),0,51.77,22,Three,It's hard to imagine a world without A Light i...
tipping-the-velvet_999/index.html,Tipping the Velvet,90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0,53.74,20,One,"""Erotic and absorbing...Written with starling ..."
soumission_998/index.html,Soumission,6957f44c3847a760,Books,£50.10,£50.10,£0.00,In stock (20 available),0,50.1,20,One,"Dans une France assez proche de la nôtre, un h..."
sharp-objects_997/index.html,Sharp Objects,e00eb4fd7b871a48,Books,£47.82,£47.82,£0.00,In stock (20 available),0,47.82,20,Four,"WICKED above her hipbone, GIRL across her hear..."
sapiens-a-brief-history-of-humankind_996/index.html,Sapiens: A Brief History of Humankind,4165285e1663650f,Books,£54.23,£54.23,£0.00,In stock (20 available),0,54.23,20,Five,From a renowned historian comes a groundbreaki...
...,...,...,...,...,...,...,...,...,...,...,...,...
alice-in-wonderland-alices-adventures-in-wonderland-1_5/index.html,Alice in Wonderland (Alice's Adventures in Won...,cd2a2a70dd5d176d,Books,£55.53,£55.53,£0.00,In stock (1 available),0,55.53,1,One,\n\n\n\n\n\n
ajin-demi-human-volume-1-ajin-demi-human-1_4/index.html,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",bfd5e1701c862ac3,Books,£57.06,£57.06,£0.00,In stock (1 available),0,57.06,1,Four,High school student Kei Nagai is struck dead i...
a-spys-devotion-the-regency-spies-of-london-1_3/index.html,A Spy's Devotion (The Regency Spies of London #1),19fec36a1dfb4c16,Books,£16.97,£16.97,£0.00,In stock (1 available),0,16.97,1,Five,"In England’s Regency era, manners and elegance..."
1st-to-die-womens-murder-club-1_2/index.html,1st to Die (Women's Murder Club #1),f684a82adc49f011,Books,£53.98,£53.98,£0.00,In stock (1 available),0,53.98,1,One,"James Patterson, bestselling author of the Ale..."


In [27]:
# сохраняем результаты
pd.DataFrame(d).T.to_csv('books.toscrape.csv')

## Умные трюки для парсинга

In [28]:
import math
a = [1,2,3,0,-1,-2,5,6]
for num in a:
    try:
        print(num, math.log(num))
    except:
        print(num, 'not in domain of log')

1 0.0
2 0.6931471805599453
3 1.0986122886681098
0 not in domain of log
-1 not in domain of log
-2 not in domain of log
5 1.6094379124341003
6 1.791759469228055


In [29]:
if 'results' not in os.listdir():
    os.mkdir('results')

In [30]:
d_short = dict(list(d.items())[:10])

for i,book_url in enumerate(tqdm(d_short)): # d_short d
    # задержка в полсекунды, чтобы сайт не злился
    time.sleep(0.5)
    try:
        url = base_url+book_url
        # добавляем фейковую информацию о том, кто делает запрос через UserAgent из fake_useragent
        response = requests.get(url, headers = {'User-Agent': UserAgent().chrome})
        tree = BeautifulSoup(response.content, 'html.parser')
        results = parse_one_book(tree)
        d[book_url].update(results)
        if i%5==0: # каждые 5 итераций будем сохранять результаты
            # сколько сейчас времени
            now = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
            # сохраняем результаты
            pd.DataFrame(d).T.to_csv(f'results/books.toscrape_{now}.csv')
    except:
        # всё, что не спарсилось, запишем, чтобы потом посмотреть
        with open('results/failures.txt', 'a') as f:
            print(book_url, file=f)

# финальные результаты
pd.DataFrame(d).T.to_csv(f'results/books.toscrape_FINAL.csv')

  0%|          | 0/10 [00:00<?, ?it/s]

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


## Еще какой-то код

### авито

In [5]:
# эту страницу сохранили в ручную
url = 'https://www.avito.ru/moskva/avtomobili/chevrolet_corvette_2014_2415607278'

response = requests.get(url, timeout=5)#, headers = {'User-Agent': UserAgent().chrome})
tree = BeautifulSoup(response.content, 'html.parser')

response

<Response [403]>

In [6]:
# открыли html и распарсили
filename = 'Chevrolet Corvette, 2014 купить в Москве _ Автомобили _ Авито.html'
with open(filename) as f:
    s = f.read()
tree = BeautifulSoup(s, 'html.parser')


In [7]:
tree.find('span', dict(itemprop="price")).get('content')

'6900000'

In [8]:
tree.find('span', {'class':"title-info-title-text"}).text

'Chevrolet Corvette, 2014'

In [9]:
params = tree.find('div', {'data-marker':'item-view/item-params'})

In [10]:
params.find_all('span')

[<span class="params-item-params-title-2gf6N">Характеристики</span>,
 <span class="css-1r29yiv">Год выпуска<!-- -->: </span>,
 <span class="css-1r29yiv">Поколение<!-- -->: </span>,
 <span class="css-1r29yiv">Пробег<!-- -->: </span>,
 <span class="css-1r29yiv">История пробега<!-- -->: </span>,
 <span class="css-1r29yiv">Владельцев по ПТС<!-- -->: </span>,
 <span class="css-1r29yiv">Состояние<!-- -->: </span>,
 <span class="css-1r29yiv">Модификация<!-- -->: </span>,
 <span class="css-1r29yiv">Тип двигателя<!-- -->: </span>,
 <span class="css-1r29yiv">Коробка передач<!-- -->: </span>,
 <span class="css-1r29yiv">Привод<!-- -->: </span>,
 <span class="css-1r29yiv">Комплектация<!-- -->: </span>,
 <span class="css-1r29yiv">Тип кузова<!-- -->: </span>,
 <span class="css-1r29yiv">Цвет<!-- -->: </span>,
 <span class="css-1r29yiv">Руль<!-- -->: </span>,
 <span class="css-1r29yiv">VIN или номер кузова<!-- -->: </span>]

In [11]:
params.find_all('li')

[<li class="params-paramsList__item-1Xeok"><style data-emotion-css="1r29yiv">.css-1r29yiv{margin:0;font-weight:normal;box-sizing:border-box;line-height:20px;font-size:16px;color:#8F8F8F;display:inline;}li.css-1r29yiv{list-style:none;}a.css-1r29yiv{-webkit-text-decoration:none;text-decoration:none;}</style><span class="css-1r29yiv">Год выпуска<!-- -->: </span>2014</li>,
 <li class="params-paramsList__item-1Xeok"><style data-emotion-css="1r29yiv">.css-1r29yiv{margin:0;font-weight:normal;box-sizing:border-box;line-height:20px;font-size:16px;color:#8F8F8F;display:inline;}li.css-1r29yiv{list-style:none;}a.css-1r29yiv{-webkit-text-decoration:none;text-decoration:none;}</style><span class="css-1r29yiv">Поколение<!-- -->: </span>C7 (2013—н. в.)</li>,
 <li class="params-paramsList__item-1Xeok"><style data-emotion-css="1r29yiv">.css-1r29yiv{margin:0;font-weight:normal;box-sizing:border-box;line-height:20px;font-size:16px;color:#8F8F8F;display:inline;}li.css-1r29yiv{list-style:none;}a.css-1r29yiv

### циан

In [12]:
# тут мы вручную сохранили некоторые страницы
url = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&metro%5B0%5D=47&offer_type=flat&room1=1&room2=1&type=4'
response = requests.get(url)
response

<Response [200]>

In [23]:
# находим их в папке help рядом с ноутбуком
pages = [x for x in os.listdir('help') if 'page' in x]
pages_ = {}
for page in pages:
    with open(f'help/{page}') as f:
        pages_[page] = BeautifulSoup(f.read())

In [27]:
# на сайте их 28
flats = pages_[page].find_all('article', {'data-name':'CardComponent'})
len(flats)

28

In [28]:
# это первая из списка
flats[0].find_all('a')[0].get('href')

'https://www.cian.ru/rent/flat/271151225/'